# 1. Preprocessing

In [1]:
from functools import reduce

import re

from nltk.tokenize import word_tokenize
from nltk.corpus import words
from bs4 import BeautifulSoup
from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy

In [2]:
tqdm.pandas()

In [3]:
df_data = pd.read_csv("../data/aapl_us_equities_news_proc_data.csv")

In [4]:
nlp = spacy.load("en_core_web_sm")

## 1.1 Consolidate text

Concats every article and title together per day

In [5]:
def concat(series):
    return reduce(lambda x, y: x + " " + y, series)


# Group by date
df_data = df_data.groupby("date").agg({"title": concat, "content": concat, "target": "first"})

# Concat title and content
df_data["text"] = df_data["title"] + " " + df_data["content"]

# Remove index
df_data = df_data.reset_index()

# Select columns
df_data = df_data[["text", "target"]]

## 1.2 Remove HTML tags

Removes HTML tags using `beautifulsoup`

In [6]:
def remove_html_tags(text):
    """
    Remove html tags from text.
    """
    soup = BeautifulSoup(text, "html.parser")

    stripped_text = soup.get_text(separator=" ")

    return stripped_text


df_data["text"] = df_data.progress_apply(lambda row : remove_html_tags(row["text"]), axis = 1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1740/1740 [00:00<00:00, 4015.00it/s]


## 1.3 Remove EOL characters

Removes characters like: `\n`, `\t`, `\r`

In [7]:
def remove_eol_characters(text):
    """
    Remove EOL characters from text.
    """
    return re.sub(r"(\n|\t|\r)", "", text)


df_data["text"] = df_data.progress_apply(lambda row: remove_eol_characters(row["text"]), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1740/1740 [00:00<00:00, 5035.19it/s]


## 1.4 Remove other characters

Removes all characters except: `a-z`, `A-Z`, `À-ȕ`

In [8]:
def remove_other_characters(text):
    """
    Remove other characters from text.
    """
    return re.sub(r"[^a-zA-ZÀ-ȕ]", " ", text)


df_data["text"] = df_data.progress_apply(lambda row: remove_other_characters(row["text"]), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1740/1740 [00:02<00:00, 797.03it/s]


## 1.5 Remove excessive spaces

Removes excessive spaces in some documents

In [9]:
def remove_excessive_spaces(text):
    """
    Remove excessive spaces from text.
    """
    return re.sub(r"\s{2,}", " ", text)


df_data["text"] = df_data.progress_apply(lambda row: remove_excessive_spaces(row["text"]), axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1740/1740 [00:01<00:00, 1424.59it/s]


## 1.6 Convert to lowercase

Converts all characters to lowercase

In [10]:
def convert_to_lowercase(text):
    """
    Convert to lowercase.
    """
    return text.lower()


df_data["text"] = df_data.progress_apply(lambda row: convert_to_lowercase(row["text"]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1740/1740 [00:00<00:00, 33429.63it/s]


## 1.7 Remove non-dictionary words, single-character words, stopwords and lemmatize

Removes words that do not occur in the english dictionary, words that consist of one character, words that are a stopword and lemmatizes the remaining words

In [11]:
dictionary = set(words.words())


def remove_non_dictionary_single_character_stopwords_lemmatize(text):
    """
    Remove non-dictionary words, single character words, stopwords and lemmatize
    text.
    """
    return " ".join(token.lemma_ for token in nlp(text) if len(token.text) > 1 and not token.is_stop and token.text in dictionary)


df_data["text"] = df_data.progress_apply(lambda row: remove_non_dictionary_single_character_stopwords_lemmatize(row["text"]), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1740/1740 [13:58<00:00,  2.08it/s]


## 1.8 Other

In [12]:
# TODO(???): enkele HTML tags (div, img, margin, etc.)
# TODO(???): Verwijder woorden die er maar een keer inzitten
# TODO(???): Verwijder woorden die veel te veel voorkomen